# ToT Example

> ToT allows LMs to perform deliberate decision making by considering multiple different reasoning paths and self-evaluating choices to
decide the next course of action, as well as looking ahead or backtracking when
necessary to make global choices.

In [19]:
# source: https://github.com/dave1010/tree-of-thought-prompting

tree_of_thoughts_prompting = '''
Simulate three brilliant, logical experts collaboratively answering a question. 

Each one verbosely explains their thought process in real-time, considering the prior explanations of others and openly acknowledging mistakes. 

At each step, whenever possible, each expert refines and builds upon the thoughts of others, acknowledging their contributions. 

They continue until there is a definitive answer to the question. 

All experts will write down the step and their thinking about the step, then share it with the group.
Then, all experts will go on to the next step, etc.
At each step all experts will score their peers response between 1 and 5, 1 meaning it is highly unlikely, and 5 meaning it is highly likely.
If any expert is judged to be wrong at any point then they leave.
After all experts have provided their analysis, you then analyze all 3 analyses and provide either the consensus solution or your best guess solution.
For clarity, your entire response should be in a markdown table. 
The question is...
'''


question = '''
Bob is in the living room.
He walks to the kitchen, carrying a cup.
He puts a ball in the cup and carries the cup to the bedroom.
He turns the cup upside down, then walks to the garden.
He puts the cup down in the garden, then walks to the garage.
Where is the ball?
'''

full_prompt = tree_of_thoughts_prompting + "\n" + question

In [20]:
from IPython.display import Markdown
from openai import OpenAI

client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content


Markdown(get_response(question))

The ball is in the garden, as Bob put the ball in the cup before going to the bedroom and then turned the cup upside down in the garden.

In [18]:
Markdown(get_response(full_prompt))

| **Step** | **Expert 1** - Explanation | **Expert 2** - Explanation | **Expert 3** - Explanation | **Scoring Round** |
|----------|---------------------------|--------------------------|--------------------------|---------------------|
| 1        | The ball is initially inside the cup in the kitchen with Bob. | Agreed with Expert 1. | Agreed with Expert 1. | 5, 5, 5 |
| 2        | Bob carries the cup with the ball to the bedroom. The ball is still in the cup. | Bob's actions create a clear progression of the cup with the ball from kitchen to bedroom. | Agreed with Expert 2. | 5, 5, 5 |
| 3        | Bob turns the cup upside down in the bedroom. The ball falls out and remains in the bedroom. | Agreed with Expert 1. | The ball falls out in the bedroom, as stated by Expert 1. | 5, 5, 5 |
| 4        | Bob goes to the garden, leaving the cup behind in the bedroom, so the ball remains in the bedroom. | Agreed with Expert 1. | The ball should be left in the bedroom as per the statements. | 5, 5, 5 |
| 5        | Bob puts the cup down in the garden without the ball in it, confirming the ball is in the bedroom. | The cup was left in the bedroom, so the ball remains there. | Agreed with Expert 2. | 5, 5, 5 |
| 6        | Bob goes to the garage. The ball is definitely in the bedroom, based on the established sequence of events. | Agreed with Expert 1. | The ball must be in the bedroom since Bob has not gone back. | 5, 5, 5 |

Based on the scoring round, the consensus solution is that the ball is in the **bedroom**.